# Dependences

In [ ]:
!pip install --upgrade google-api-python-client
!pip install bitsandbytes>=0.39.0
!pip install git+https://github.com/huggingface/transformers.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install tiktoken
!pip install torch
!pip install scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 47.9 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.1/182.1 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.5/120.5 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.9/83.9 kB 8.4 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-uaf34nd3
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-uaf34nd3
  Resolved https://github.com/huggingface/transformers.git to commit 30ed3adf474aaf2972ab56f5624089bc24a6adf3
  Installing bu

In [1]:
!nvidia-smi

Thu Jul 13 13:43:54 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.41.03              Driver Version: 530.41.03    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3070         Off| 00000000:01:00.0 Off |                  N/A |
|  0%   55C    P8               16W / 220W|     15MiB /  8192MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [7]:
!free

              total        used        free      shared  buff/cache   available
Mem:      181108932    10473132    73337164     5609268    97298636   163382608
Swap:             0           0           0


# Imports

In [1]:
import os
import torch
import time
import datetime
import sys
import traceback
import gc

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

# Loading the model

In [2]:
model_name = 'legendhasit/xgen-7b-8k-inst-8bit'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/linagora/.local/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda113.so
CUDA SETUP: CUDA runtime path found: /home/linagora/anaconda3/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 113
CUDA SETUP: Loading binary /home/linagora/.local/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda113.so...


/home/linagora/.local/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/home/linagora/anaconda3/lib/libcudart.so'), PosixPath('/home/linagora/anaconda3/lib/libcudart.so.11.0')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
!pip install huggingface_hub["cli"]
!huggingface-cli delete-cache

# Inference

In [7]:
def mkdir(folder_path):
    try:
        os.mkdir(folder_path)
    except FileExistsError:
        pass

# Define prompt template
# ==========================================================================================
PROMPT_TEMPLATE = """A chat between a curious human and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the human's questions.

### Human: Summarize the following text.

<UI> Uh, making a profit of fifty million Euros.
<ID> Alright so twenty five.
<ME> Mm 'kay.
<UI> So, it's go gonna have to be be pretty damn trendy.
<ID> So yeah, I've The only the only remote controls I've used usually come with the television, and they're fairly basic.
<UI> Yeah.
<ME> Mm-hmm.
<UI> Yeah.
<ID> So uh Mm.
<UI> Yeah, I was thinking that as well, I think the the only ones that I've seen that you buy are the sort of one for all type things where they're, yeah.
<ME> Yeah the universal ones.
<ME> Yeah.
<UI> So presumably that might be an idea to put into.
<ID> But but to sell it for twenty five you need a lot of neat features.
<PM> Slim.
<ID> For sure.
<ME> Yeah.
<UI> Yeah, yeah.
<UI> Uh 'cause I mean, what uh twenty five Euros, that's about I dunno, fifteen Pounds or so?
<ME> Mm-hmm, it's about that.
<UI> And that's quite a lot for a remote control.
<ID> Yeah, yeah.
<ME> Mm.
<ME> Um well my first thoughts would be most remote controls are grey or black.
<ME> As you said they come with the TV so it's normally just your basic grey black remote control functions, so maybe we could think about colour?
<UI> Uh-huh.
<UI> Mm-hmm.
<ME> Make that might make it a bit different from the rest at least.
<ME> Um, and as you say, we need to have some kind of gimmick, so um I thought maybe something like if you lose it and you can whistle, you know those things?
<UI> Okay.
<UI> The the keyrings, yeah yeah.
<ME> Because we always lose our remote control.
<ID> Right.
<UI> Okay, that's cool.
<PM> Uh yeah uh, being as a Marketing Exper Expert I will like to say like before deciding the cost of this remote control or any other things we must see the market potential for this product like what is the competition in the market?
<PM> What are the available prices of the other remote controls in the prices?
<PM> What speciality other remote controls are having and how complicated it is to use these remote controls as compared to other remote controls available in the market.
<UI> Okay.
<PM> So before deciding or before finalising this project, we must discuss all these things, like and apart from this, it should be having a good look also, because people really li uh like to play with it when they are watching movies or playing with or playing with their CD player, MP three player like any electronic devices.
<ME> Okay.
<ME> Mm.
<ME> Mm-hmm.
<ME> Mm-hmm.
<PM> They really want to have something good, having a good design in their hands, so, yes, all this.
<ME> Yeah.
<UI> Okay.
<UI> 'Kay.
<ID> Uh, what do we think a What do we think a good size would be for this?
<UI> So, we're looking for 'Kay.
<UI> We're Sorry, carry on.
<ID> 'Cause I I know as you add more buttons to the remote it sometimes gets so big and clunky and there's just like a hundred buttons on it, or you could have a really small slim one but then you could lose it easily.
<ME> Yeah.
<UI> Mm-hmm.
<UI> Mm-hmm.
<ME> Yeah.
<ME> Then you lose it, yeah.
<UI> Okay.
<ME> Kind of um, maybe more like a PDA kind of, just hand held, like, 'cause Yeah.
<UI> For for uh remember we're trying to make it for twelve Euros fifty.
<ME> No, I wasn't, no sorry I wasn't thinking of the screen of like a PDA but Okay.
<UI> Okay well right we'll have to um I'll we're k having another meeting in half an hour so um we should all look into a bit uh, oh actually, no, we'll allocate.
<UI> So you do the looking around at other remote controls.
<PM> Yeah.
<UI> Um, if you could maybe come up with sort of shapes and suggested shades or whatever, and you could look into um basically how how it's made IE like how you make it all in one, how what sort of materials are available to you whatever.
<UI> And obviously, other instructions will come from the personal coach.
<ID> Right.
<UI> Which will probably just usurp what I said so Shapes and colours and um basically how to make it attractive.
<ME> So you want me to look at shapes and everything you said?
<ME> Yep.
<ME> Okay.
<UI> Uh.
<ME> Mm-hmm.
<UI> And you look at competition and design.
<PM> Yep.
<UI> Cool.
<ME> Okay.
<ID> Okay.
<UI> So we have uh Um.
<ID> Wait for emails?
<PM> Uh.
<ID> Hmm.
<UI> Okay, groovy.
<UI> And no doubt we'll get um Sorry.
<ME> Oh no, .
<ME> Sorry it's okay.
<UI> We'll get um warnings for next meetings as well.
<ME> Okay, cool.
<UI> Okay.
<UI> I shall I can't imagine these are worth much.
<UI> Okay.
<PM> Hmm.
<UI> Fashion into electronic.
<UI> Okay.

### Assistant: The group discussed their initial ideas about the features that they wanted to integrate into the design. They discussed making a universal remote with a locator function. They also discussed the shape and the number of functions in the main interface. The Project Manager instructed the Marketing Expert to examine competitors' remotes, the User Interface Designer to research possible shapes and colors, and the Industrial Designer to research possible materials and the necessary internal components of the device.

### Human: Summarize the following text.

<ID> So welcome.
<ID> The first kick-off meeting.
<ID> What shall we do?
<ID> First the opening, then the rest.
<ID> What are we going to do.
<ID> We m have to make a new remote control.
<ID> It has to be original, trendy and user-friendly.
<ID> So we will get back th on that.
<ID> First we have to make a functional design.
<ID> After that we have to make a conceptual design, and then after that a detailed design.
<ID> So we'll discuss that later.
<ID> First we have a look at.
<ID> So first to we have to make a small painting.
<ID> What have do we have to do.
<ID> First you can save the documents.
<ID> We have to do that every time we make something.
<ID> You can print it.
<ID> No.
<ID> And we have to use the pen and the eraser.
<ID> So Now.
<ID> We all have to use this one.
<ID> You have to make your own favourite animal.
<ID> So I'll make an example.
<ME> Yep.
<ID> First don't touch that things.
<ID> You can use the pen.
<ID> And then you can make um something.
<UI> Nice.
<ID> Um you can change some things.
<ID> Um format, line, and change it.
<ID> And you can change the colour.
<UI> An elephant.
<ID> So that's it.
<ID> So So and after it you have to save it.
<ME> Okay.
<ID> Now we can make a new one.
<ID> You have to paint now.
<ME> Oh.
<ID> So you're next.
<UI> 'Kay.
<ME> Well we will try.
<ME> Where it going?
<PM> Hmm.
<PM> That's uh strange.
<ID> What is going on?
<UI> pop-ups.
<ID> What are you What?
<ME> Hmm.
<UI> What is this, Pictionary.
<ME> Uh Mm.
<ID> Um Is a It is a It is a A duck.
<UI> Uh a bird.
<UI> Bird.
<ME> So Now save?
<UI> Yeah.
<ID> Yes.
<ID> Hmm.
<ME> Now uh blank?
<ID> Blank, yes.
<PM> Yeah.
<ME> Yeah.
<ID> Okay next one.
<PM> Okay.
<PM> Let's try this.
<UI> Whoo.
<PM> Uh Um.
<ME> Yeah, yeah.
<PM> Mm-hmm.
<PM> Mm.
<UI> Oh not.
<UI> Oh.
<ID> Oh.
<UI> Okay.
<UI> Okay.
<UI> Yeah.
<UI> No problem.
<UI> Shit happens.
<ME> I'm not getting anything uh on my screen now.
<ME> Okay.
<UI> A parrot.
<ME> Wow.
<UI> Ish.
<ME> Oh.
<UI> He did it before.
<PM> Uh No, no.
<PM> Yeah.
<PM> Okay.
<ME> Nice.
<UI> Oh.
<ID> Very good.
<PM> Uh blank.
<UI> Thank you.
<ID> Okay.
<ID> Very good.
<ID> So um you can always go back.
<ID> So That's it.
<ID> So that was two.
<ID> Now next.
<ID> The budget.
<ID> The b Uh we will sell the t at twenty five Euros.
<ID> And we have only twenty of twelve and a half Euro to make it.
<ID> So now we have to think about what we will make.
<ID> First I wanna hear from you.
<ID> Uh what are your experiences with remote controls.
<ID> So F first.
<UI> Uh I will start.
<UI> Uh Big one, they are uh not easy to use.
<UI> Um I have one set and uh a remote control, when I dropped it, uh it broke.
<UI> So that won't be uh our goal, I think.
<PM> No.
<UI> And uh g big buttons, m uh that's easier to use than uh I think.
<UI> Not all the small buttons, you don't know Big buttons, positive.
<ID> Is this positive or negative, that uh big buttons?
<ID> Positive.
<UI> All all small buttons like when you have uh like a hundred buttons on your remote control, you won't know what they're working for.
<ID> Okay.
<ID> What are your experiences?
<PM> Uh well I think the the the goal of a remote control is that it's it it has an influence on the TV set.
<PM> And that it controls the channels and the the volume.
<ID> Mm.
<PM> And uh I I I think it's positive if there's a a LED uh uh a LED on the corner of the of the remote.
<PM> So that you know it s it still has batteries on it in it.
<PM> And that if you push the button the LED uh gives a light, and uh and you see that it's working.
<PM> And uh yeah.
<PM> Uh Yeah, but No no no.
<ID> So and do they always have that?
<PM> But I my my experience is that it it it's convenient to have that.
<ID> It's easy to you.
<PM> Yeah.
<ID> Okay.
<PM> Yeah.
<ID> 'Kay.
<ME> Uh at home we have a TV, a video uh recorder, a DVD player, and a satellite receiver.
<ME> We have uh four distinctive remote controls for that.
<UI> Thank you.
<ME> That's not really ea easy.
<UI> Help also.
<ME> So it would be nice if we have one for all.
<UI> Thank you.
<ME> And we also had a remote control for our radio set.
<ME> But um i it it had a lot of buttons on it, and you didn't know which one was what.
<ME> And it was uh uh v not easy to use.
<ME> So we n barely used it.
<ID> Okay so they have too much.
<ID> So next.
<PM> Hmm.
<ID> For our own remote control we have to think how do we make it.
<ID> So what ideas do you have for it, for the new remote control?
<ID> What what does it have to have?
<UI> The weight.
<UI> Not not too heavy.
<ID> Not too heavy.
<UI> Not much buttons.
<ID> Yes.
<ID> Yeah.
<UI> Bust-free.
<UI> That when you drop it, it won't break.
<UI> Like uh some kind of rubber on it.
<UI> Or hard uh hard plastic.
<UI> Uh buttons not too small.
<UI> Uh something like when you uh lose your uh remote control, sometimes it happen.
<ID> Yes.
<UI> Uh it between the couch and you can't find it.
<UI> When you push a but a button on the TV, then you hear some uh some sort of bleep.
<ID> Like a phone.
<UI> And then you uh, hey there there's remote control.
<ME> Yeah.
<ID> Okay.
<PM> Yeah.
<ID> So, that's.
<UI> Next.
<PM> Yeah well that's that are good ideas.
<PM> Uh Yeah well the LED on the corner, that that indicates that it's working.
<PM> If you push a button.
<PM> Um Yeah.
<PM> And looking on the budget, not too expensive uh material.
<PM> So probably plastic or something.
<PM> Uh Mm no.
<ID> Okay.
<ME> Yeah I think it uh from a marketing point of view, it also has to look nice.
<ME> Or you won't sell it.
<ID> Yes.
<ME> And um yeah uh on our website we can see what products we already have.
<ME> And it should work with as many uh as possible of them.
<ID> Okay.
<ID> This is It has to be compatible with other things.
<ME> Yes.
<ID> Okay.
<UI> I have one more idea.
<UI> Just popped up.
<ID> Yes?
<UI> Uh it it won't take a lot of batteries.
<UI> So you don't won't have to change the batteries uh once a week or uh once every two weeks.
<ID> No battery use.
<ID> So more ideas?
<ID> No okay.
<ID> It's only the first ideas.
<ID> So uh what are we going to do now is Next meeting is in half an h hour.
<ID> Uh Okay.
<ID> Next meeting, half an hour.
<ID> Um, what you have to do.
<ID> Well look on your.
<ID> And Next instructions you'll get in your email.
<ID> So This is the first meeting.
<ID> See you later in half an hour.
<PM> Yes.
<UI> Okay.
<ME> Okay.
<UI> Thank you.

### Assistant: For the first meeting, the task of designing a remote control was briefly introduced along with the plan for the subsequent meetings. The group then drew animals to practise using the drawing platform. They discussed their likes and dislikes regarding current remote controls, including ease of use, multiple systems and power indicators. They then offered suggestions as to what they would like from their remote. They would like the remote to be durable, for it to include a device to help find it when lost and not use too many batteries.

### Human: Summarize the following text.

{text}

### Assistant:"""

# Load samples from dataset
# ==========================================================================================
samples = os.listdir('input/texts')
samples = [sample for sample in samples if sample.endswith('.txt')]
samples.sort()
n_samples = len(samples)
print('-- Found', n_samples, 'samples:', samples)

-- Found 12 samples: ['sample_10_9852.txt', 'sample_11_10498.txt', 'sample_12_11166.txt', 'sample_1_4693.txt', 'sample_2_11567.txt', 'sample_3_12019.txt', 'sample_4_13389.txt', 'sample_5_1343.txt', 'sample_6_10232.txt', 'sample_7_11826.txt', 'sample_8_7292.txt', 'sample_9_4578.txt']


In [3]:
MAX_CHARACTER_CHUNK_SIZE = 2000 * 3

def chunkize(text):
    """
    Greedy implementation of a dialogue transcript chunking algorithm. This method returns a list of transcript chunks.
    - It priorities stability over performance. There is a set maximum chunk size for LLM inference stability. Really long utterances bypass this limit.
    - It guarantees the cuts are made at utterance ends.

    !! WARNING !! The maximum size is in characters, not tokens. A good upper bound shall be used as there are
    no clear and easy correspondance between characters and tokens, as it obviously depends on the model tokenizer.

    Dialogue format recognized (used in the HF AMI dataset):
    ----------------------
    <A> Hello!
    <B> Hey there
    <A> What have you been up to
    <B> Not much honestly
    ----------------------
    """
    
    chunks = [] # Final list of transcript chunks. This makes up the loop invariant
    utterances = text.split('\n') # Transcript is split into sentences
    current_chunk = ''

    # While there is still an utterance to process
    while len(utterances) > 0:
        utterance = utterances.pop()

        # Add to current chunk and proceed to next
        if len(current_chunk) + len(utterance) <= MAX_CHARACTER_CHUNK_SIZE:
            if len(current_chunk) > 0:
                current_chunk += '\n'
            current_chunk += utterance
        
        # Utterance is larger than maximum chunk size
        elif len(current_chunk) == 0:
            chunks.append(current_chunk)
            chunks.append(utterance)
            current_chunk = ''

        # Current chunk is big enough, append to list and create new one
        else:
            chunks.append(current_chunk)
            current_chunk = utterance

    if len(current_chunk) > 0:
        chunks.append(current_chunk)

    return chunks

In [8]:
# Tests for chunking algorithm
import os

sample_file_path = 'input/texts/' + samples[0]
sample_file = open(sample_file_path, 'r', encoding='utf-8')
sample = sample_file.read()
sample_file.close()

chunks = chunkize(sample)
for chunk in chunks:
    print('-----------------------------------------')
    print(chunk)

-----------------------------------------
<ME> Okay, see you.
<UI> Okay, so you can I guess we'll see you for lunch in a sec?
<ME> Cool.
<UI> Right on time.
<UI> Okay?
<ME> Okay.
<UI> No, we have we have after lunch we have thirty minutes to ourselves to prepare, so that's fine, w before lunch we just have to complete the questionnaire and some sort of summary.
<ME> Sorry, but um the next meeting um are we going to have it um right after lunch or shall we prepare our To prepare, okay, yeah, that's good.
<ME> Hmm.
<UI> Components, yeah.
<ID> Con components, oh.
<ID> Wha what was it again that I was supposed to look into?
<ME> Uh something conceptual, yeah.
<UI> You'll be looking you'll be looking at the user interface concept, on something conceptual and you're watching trends to see how we go and surely voice recognition'll fall off the map or something that um we'll keep keep our options op hmm?
<ID> I guess I'll find out.
<UI> Yeah.
<ME> Yeah.
<UI> Whatever that means.
<UI> Yeah?
<ID

In [15]:
def inference_sample(sample):

    # Current summary is the concatenation of all the chunk sub-summaries 
    current_summary = ''

    # Split into chunks
    for chunk in chunkize(sample):

        # Create prompt for this chunk
        prompt = PROMPT_TEMPLATE.format(text=chunk)

        # Sample one sub-input
        input_ids = tokenizer(chunk, return_tensors="pt").to('cuda')
        output_ids = model.generate(**input_ids, do_sample=True, max_new_tokens=2048, top_k=100, eos_token_id=50256, temperature=0.3)
        output = tokenizer.decode(sample[0]).strip()

        # Add to summary
        if len(current_summary) > 0:
            current_summary += '\n\n'
        current_summary += output
    
    return current_summary

In [7]:
# Inference
# ==========================================================================================
initial_time = time.time()
skipped_samples = 0

mkdir('intermediate')

print('Starting computation...')

# For each sample in dataset
for sample_n in range(n_samples):

    # Estimate completion and time.
    cur_samples = sample_n - skipped_samples
    tot_samples = n_samples - skipped_samples
    progress = cur_samples / tot_samples
    pct = round(progress * 100, 1)
    print('Prompting on sample N' + str(sample_n + 1) + '/' + str(n_samples))
    print('-- Completion: ' + str(pct) + '%')
    if cur_samples > 0:
        approx_total = (time.time() - initial_time) / cur_samples * tot_samples
        approx_remaining = approx_total * (1 - progress)
        print('-- Estimated Remaining Time: ' + str(datetime.timedelta(seconds=int(approx_remaining))) + ' (total ' + str(datetime.timedelta(seconds=int(approx_total))) + ')')
    
    # Read sample and generate prompt
    sample_file_path = 'input/texts/' + samples[sample_n]
    sample_file = open(sample_file_path, 'r', encoding='utf-8')
    sample = sample_file.read()
    sample_file.close()
    
    # Find target file
    target_file_path = 'intermediate/' + samples[sample_n]
    if os.path.isfile(target_file_path):
        print('-- Found intermediate result file \'' + target_file_path + '\', skipped.')
        skipped_samples += 1
        continue

    try:

        # Inference
        output = inference_sample(sample)

        # Save answer in file
        target_file = open(target_file_path, 'w', encoding='utf-8')
        target_file.write(output)
        target_file.close()
    
    except:
        print('Could not compute prompt:')
        print(sample)
        traceback.print_exc()

delta = time.time() - initial_time
print('Done! Took', datetime.timedelta(seconds=int(delta)), 'seconds')

Starting computation...
Prompting instruction N1/6 on sample N1/10
-- Completion: 0.0%
-- Found intermediate result file 'intermediate/1_1.txt', skipped.
Prompting instruction N1/6 on sample N2/10
-- Completion: 0.0%
-- Found intermediate result file 'intermediate/1_2.txt', skipped.
Prompting instruction N1/6 on sample N3/10
-- Completion: 0.0%
-- Found intermediate result file 'intermediate/1_3.txt', skipped.
Prompting instruction N1/6 on sample N4/10
-- Completion: 0.0%
-- Found intermediate result file 'intermediate/1_4.txt', skipped.
Prompting instruction N1/6 on sample N5/10
-- Completion: 0.0%
-- Found intermediate result file 'intermediate/1_5.txt', skipped.
Prompting instruction N1/6 on sample N6/10
-- Completion: 0.0%
-- Found intermediate result file 'intermediate/1_6.txt', skipped.
Prompting instruction N1/6 on sample N7/10
-- Completion: 0.0%
-- Found intermediate result file 'intermediate/1_7.txt', skipped.
Prompting instruction N1/6 on sample N8/10
-- Completion: 0.0%
-- 

# Calculs de scores

In [2]:
!pip install rouge_score rouge
!pip install evaluate
!pip install bert-score
!pip install sacrebleu
!pip install nltk

  Preparing metadata (setup.py) ... done
  Using cached absl_py-1.4.0-py3-none-any.whl (126 kB)
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=333555242f4e464ad2f1466233462741b2f13d4d85af3a6bfe866ca7f1f269f6
  Stored in directory: /home/linagora/.cache/pip/wheels/3e/94/5c/7ff8a51c53c1bbc8df4cac58aa4990ffbc6fa203e9f0808fdd
Successfully built rouge_score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 29.0 MB/s eta 0:00:00
  Using cached multiprocess-0.70.14-py310-none-any.whl (134 kB)
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)
  Using cached xxhash-3.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (212 kB)
  Using cached pyarrow-12.0.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (38.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 8.0 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 12.6 MB/s eta 0:00:00


In [2]:
import evaluate
import os


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/linagora/.local/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda113.so
CUDA SETUP: CUDA runtime path found: /home/linagora/anaconda3/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 113
CUDA SETUP: Loading binary /home/linagora/.local/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda113.so...


/home/linagora/.local/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/home/linagora/anaconda3/lib/libcudart.so'), PosixPath('/home/linagora/anaconda3/lib/libcudart.so.11.0')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)


In [4]:
# Methods and variables
# ==========================================================================================
print('Starting scores computation...')
bleu = evaluate.load("bleu")
rouge = evaluate.load('rouge')
bertscore = evaluate.load('bertscore')

STORAGE_FILE_NAME = 'scores'

# Used to process out n-shot examples
def find_nth_occurrence(s, sub, n):
    count = s.count(sub)
    if count < n:
        return -1
    else:
        index = -1
        for i in range(n):
            index = s.find(sub, index+1)
        return index

# Script itself
# ==========================================================================================

# Find output file for CSV scores
mkdir('output')
storage_file = open('output/' + STORAGE_FILE_NAME + '.csv', 'w', encoding='utf-8')
storage_file.write('path;rouge2;rougel;bertscore\n')

target_file_paths = []
references = []
predictions = []

# For each sample in dataset
for sample_n in range(n_samples):

    # Find generated summary
    target_file_path = 'intermediate/' + samples[sample_n]
    if not os.path.isfile(target_file_path):
        print('-- Found no intermediate result file \'' + target_file_path + '\', skipped.')
        continue

    # Read sample and generate prompt -> Keep summary
    summary_file_path = 'input/summaries/' + samples[sample_n]
    summary_file = open(summary_file_path, 'r', encoding='utf-8')
    references.append(summary_file.read())
    summary_file.close()

    # Access generated summary
    target_file = open(target_file_path, 'r', encoding='utf-8')
    prediction = target_file.read()
    target_file.close()

    # Process answer
    separator = "### Assistant:"
    cut_index = find_nth_occurrence(prediction, separator, 3) + len(separator)
    prediction = prediction[cut_index:]
    if prediction[0] == " ": # Enlever l'espace devant
        prediction = prediction[1:]
    prediction = prediction[:-len("<|endoftext|>") - 2]

    # Add prediction
    target_file_paths.append(target_file_path)
    predictions.append(prediction)

# Calculate metrics
result_rouge = rouge.compute(predictions=predictions, references=references, use_aggregator=False)
result_bertscore = bertscore.compute(predictions=predictions, references=references, lang='fr', rescale_with_baseline=True, verbose=True)

# Write to csv
# Forget about BLEU...
# Format: PATH | ROUGE2 | ROUGEL | BERTScore
for i in range(len(target_file_paths)):
    ligne = target_file_paths[i]
    ligne += ';' + str(result_rouge['rouge2'][i]) + ";" + str(result_rouge['rougeL'][i])
    #ligne += ";" + str(result_bleu['bleu'])
    ligne += ";" + str(result_bertscore['f1'][i])

    storage_file.write(ligne + '\n')

storage_file.close()
print('Done!')

Starting scores computation...
calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 279482.79 seconds, 0.00 sentences/sec
Done!
